In [1]:
import pandas as pd
import gc

In [2]:
import feather

In [3]:
import os
os.listdir('../data/')

['clicks_test.csv.zip',
 'clicks_train.csv.zip',
 'documents_categories.csv.zip',
 'documents_entities.csv.zip',
 'documents_meta.csv.zip',
 'documents_topics.csv.zip',
 'events.csv.zip',
 'promoted_content.csv.zip',
 'sample_submission.csv.zip']

In [4]:
data_path = '../data/'
click_train=pd.read_csv(data_path+'clicks_train.csv.zip',chunksize=100000)

In [5]:
## 取前10w个数据吧
c = 0 
df_click_train=None
for item  in click_train:
    df_click_train=item
    break

In [6]:
df_click_train.clicked.sum()

19587

In [7]:
df_click_train.shape

(100000, 3)

In [8]:
click_test = pd.read_csv(data_path+'clicks_test.csv.zip',chunksize=100000)

In [9]:
df_click_test=None
for item in click_test:
    df_click_test = item
    break

In [10]:
df_click_test.shape

(100000, 2)

In [11]:
df_click_test.head()

,display_id,ad_id
0,16874594,66758
1,16874594,150083
2,16874594,162754
3,16874594,170392
4,16874594,172888


In [12]:
display_train=set(df_click_train.display_id)

In [13]:
display_test=set(df_click_test.display_id)

In [14]:
a = display_train.intersection(display_test)

In [15]:
ad_train=set(df_click_train.ad_id)
ad_test=set(df_click_test.ad_id)

In [16]:
df_events=pd.read_csv(data_path+'events.csv.zip',chunksize=100000)


In [17]:
import tqdm
c = 0

for item in tqdm.tqdm(df_events):
    if c==0:
        c+=1
        df_train_events=item
        df_test_events=item
    item1 = item.loc[item.display_id.isin(df_click_train.display_id),:]
    df_train_events = df_train_events.append(item1)
    item2 = item.loc[item.display_id.isin(df_click_test.display_id),:]
    df_test_events = df_test_events.append(item2)
    

232it [01:04,  3.58it/s]


In [18]:
df_total_events=pd.concat([df_train_events,df_test_events],axis=0)

In [19]:
df_total_events.shape

(239221, 6)

In [20]:
def paths(tokens):
    all_paths = ['_'.join(tokens[0:(i+1)]) for i in range(len(tokens))]
    return ' '.join(all_paths)

def unwrap_geo(geo):
    geo = geo.split('>')
    return paths(geo)

In [21]:
df_total_events.geo_location.fillna('', inplace=True)

In [22]:
geo_str = df_total_events.geo_location.apply(unwrap_geo)


In [23]:
ts = (df_total_events.timestamp + 1465876799998) / 1000
df_total_events.timestamp = pd.to_datetime(ts, unit='s')

In [24]:
dt = df_total_events.timestamp.dt

dow = 'dow_' + dt.dayofweek.astype('str')
hours = 'hour_' + dt.hour.astype('str')
dow_hour = 'dow_hour_' + dt.dayofweek.astype('str') + '_' + dt.hour.astype('str')

display_str = 'u_' + df_total_events.uuid + ' ' + \
              'd_' + df_total_events.document_id.astype('str') + ' ' + \
              'p_' + df_total_events.platform.astype('str') + ' ' +  \
              dow + ' ' + hours + ' ' + dow_hour + ' ' + \
              geo_str

In [25]:
df_events_processed = pd.DataFrame()
df_events_processed['display_id'] = df_total_events.display_id
df_events_processed['display_str'] = display_str

In [26]:
df_promoted = pd.read_csv(data_path+"promoted_content.csv.zip")

In [27]:
ad_string = 'addoc_' + df_promoted.document_id.astype('str') + ' ' \
            'campaign_' + df_promoted.campaign_id.astype('str') + ' ' \
            'adv_' + df_promoted.advertiser_id.astype('str') 

df_promoted_processed = pd.DataFrame()
df_promoted_processed['ad_id'] = df_promoted.ad_id
df_promoted_processed['promoted_ad_str'] = ad_string

In [28]:
ad_to_idx = dict(zip(df_promoted_processed.ad_id, df_promoted_processed.index))
display_to_idx = dict(zip(df_events_processed.display_id,df_events_processed.index))

In [29]:
# processing data in batches

def prepare_batch(batch):
    batch = batch.reset_index(drop=1)
    
    promoted_idx = batch.ad_id.apply(ad_to_idx.get)
    promoted_ad_str = df_promoted_processed.promoted_ad_str.iloc[promoted_idx]

    ## 这行代码有问题吧？？？
    display_idx = batch.display_id.apply(display_to_idx.get)
#     display_str = df_events_processed.display_str.iloc[batch.display_id - 1]
    display_str = df_events_processed.display_str.loc[display_idx]
    
    promoted_ad_str.reset_index(drop=True, inplace=True)
    display_str.reset_index(drop=True, inplace=True)

    batch['ad_display_str'] = promoted_ad_str + ' ' + display_str
    return batch

In [41]:
def append_to_csv(batch, csv_file):
    props = dict(encoding='utf-8', index=False)
#     if not os.path.exists(csv_file):
    batch.to_csv(csv_file, **props)
#     else:
#         batch.to_csv(csv_file, mode='a', header=False, **props)

In [42]:
def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

def chunk_dataframe(df, n):
#     for i in range(0, len(df), n):
    for i in range(0, 2000000,n):
        yield df.iloc[i:i+n]

In [43]:
## 每次展示都有一个唯一的display_id，所以display_id肯定不会重复

In [44]:

batch = prepare_batch(df_click_train)
batch['fold']=0
total_index=list(range(df_click_train.shape[0]))
import numpy as np
sam_ind = np.random.choice(total_index,int(len(total_index)/2),replace=False)
batch.loc[batch.iloc[sam_ind,].index,'fold']=1

In [45]:
append_to_csv(batch, './processed/svm_features_train.csv')

In [46]:
batch = prepare_batch(df_click_test)
append_to_csv(batch, './processed/svm_features_test.csv')